# Lab 1.2
### Імпорт бібліотек

In [56]:
import pandas as pd
import os
# import numpy as np
import sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler


# Task 1

### Завантаження і очистка данних 

In [55]:
path = r"C:/Specialni_Rozdily_Prog/specprograming/household_power_consumption.txt"

df = pd.read_csv(path, sep=";", low_memory=False)
output_path = r"C:/Specialni_Rozdily_Prog/specprograming/household_power_consumption_converted.csv"
df.to_csv(output_path, sep=",", index=False)

print(df.head(10))

         Date      Time Global_active_power Global_reactive_power  Voltage  \
0  16/12/2006  17:24:00               4.216                 0.418  234.840   
1  16/12/2006  17:25:00               5.360                 0.436  233.630   
2  16/12/2006  17:26:00               5.374                 0.498  233.290   
3  16/12/2006  17:27:00               5.388                 0.502  233.740   
4  16/12/2006  17:28:00               3.666                 0.528  235.680   
5  16/12/2006  17:29:00               3.520                 0.522  235.020   
6  16/12/2006  17:30:00               3.702                 0.520  235.090   
7  16/12/2006  17:31:00               3.700                 0.520  235.220   
8  16/12/2006  17:32:00               3.668                 0.510  233.990   
9  16/12/2006  17:33:00               3.662                 0.510  233.860   

  Global_intensity Sub_metering_1 Sub_metering_2  Sub_metering_3  
0           18.400          0.000          1.000            17.0  
1      

#### Конвертуємо числові колонки

In [57]:
df["Global_active_power"] = pd.to_numeric(df["Global_active_power"], errors="coerce")
df["Global_intensity"] = pd.to_numeric(df["Global_intensity"], errors="coerce")
df["Sub_metering_1"] = pd.to_numeric(df["Sub_metering_1"], errors="coerce")
df["Sub_metering_2"] = pd.to_numeric(df["Sub_metering_2"], errors="coerce")
df["Sub_metering_3"] = pd.to_numeric(df["Sub_metering_3"], errors="coerce")

#### Створюємо datetime

In [58]:
df["Datetime"] = pd.to_datetime(df["Date"] + " " + df["Time"], dayfirst=True,errors='coerce')

#### Вибір числових колонок і їх очистка від '?' -> NaN

In [59]:
cols = ["Global_active_power", "Global_reactive_power", 
            "Voltage", "Global_intensity", 
            "Sub_metering_1", "Sub_metering_2", "Sub_metering_3"]
for col in cols:
    df[col] = pd.to_numeric(df[col], errors="coerce") 

#### створення “скейлерів” (масштабувальників) із бібліотеки scikit-learn, для масштабування числових даних

In [60]:
scaler_minmax = MinMaxScaler()
scaler_standard = StandardScaler()

# Task 2

### Sample 1

In [61]:
maxpowers_df=df[df["Global_active_power"] > 5]
print(maxpowers_df.head(7))

          Date      Time  Global_active_power  Global_reactive_power  Voltage  \
1   16/12/2006  17:25:00                5.360                  0.436   233.63   
2   16/12/2006  17:26:00                5.374                  0.498   233.29   
3   16/12/2006  17:27:00                5.388                  0.502   233.74   
11  16/12/2006  17:35:00                5.412                  0.470   232.78   
12  16/12/2006  17:36:00                5.224                  0.478   232.99   
13  16/12/2006  17:37:00                5.268                  0.398   232.91   
20  16/12/2006  17:44:00                5.894                  0.000   232.69   

    Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  \
1               23.0             0.0             1.0            16.0   
2               23.0             0.0             2.0            17.0   
3               23.0             0.0             1.0            17.0   
11              23.2             0.0             1.0           

### Sample 2

In [62]:
voltage_df = df[
    (df["Global_intensity"].isin([19,20])) &
    ((df["Sub_metering_1"] + df["Sub_metering_2"]) > df["Sub_metering_3"]+ df["Sub_metering_1"])
]
print(voltage_df.head(7))

             Date      Time  Global_active_power  Global_reactive_power  \
45     16/12/2006  18:09:00                4.464                  0.136   
944    17/12/2006  09:08:00                4.762                  0.088   
945    17/12/2006  09:09:00                4.506                  0.088   
5151   20/12/2006  07:15:00                4.664                  0.232   
5152   20/12/2006  07:16:00                4.774                  0.228   
11425  24/12/2006  15:49:00                4.510                  0.482   
11426  24/12/2006  15:50:00                4.806                  0.494   

       Voltage  Global_intensity  Sub_metering_1  Sub_metering_2  \
45      234.66              19.0             0.0            37.0   
944     237.44              20.0             0.0            38.0   
945     237.25              19.0             0.0            38.0   
5151    240.25              20.0             0.0             2.0   
5152    239.79              20.0             0.0           

### Sample 3

In [63]:
sample_df = df.sample(n=500_000, replace=False, random_state=42)
print("Перевірка розмірів sample_df: ",sample_df.shape) 

mean_values = sample_df[["Sub_metering_1", "Sub_metering_2", "Sub_metering_3"]].mean()

print("Середнє споживання для трьох груп (Вт·год):")
print(mean_values)

Перевірка розмірів sample_df:  (500000, 10)
Середнє споживання для трьох груп (Вт·год):
Sub_metering_1    1.123054
Sub_metering_2    1.299983
Sub_metering_3    6.465535
dtype: float64


### Sample 4

#### dataframe для роботи приладів 2 групи після 18:00 і споживання понад 6 кВт

In [64]:
# після 18:00 і споживання понад 6 кВт
df_evening = df[(df["Datetime"].dt.hour >= 18)&
                (df["Global_active_power"] > 6)]

#  основне споживання за 2 групу
df_evening2 = df_evening[
    df_evening["Sub_metering_2"] >
    df_evening[["Sub_metering_1","Sub_metering_3"]].max(axis=1)]

####  вибір рядків (кожен третій  із першої половини та кожен четвертий із другої половини)

In [65]:
half = len(df_evening2) // 2

first_half = df_evening2.iloc[:half].iloc[::3]     
second_half = df_evening2.iloc[half:].iloc[::4]

final_sample = pd.concat([first_half, second_half])

print(final_sample.head())

print(f"Кількість вибраних рядків: {len(final_sample)}")

             Date      Time  Global_active_power  Global_reactive_power  \
41     16/12/2006  18:05:00                6.052                  0.192   
44     16/12/2006  18:08:00                6.308                  0.116   
17494  28/12/2006  20:58:00                6.386                  0.374   
17498  28/12/2006  21:02:00                8.088                  0.262   
17501  28/12/2006  21:05:00                7.230                  0.152   

       Voltage  Global_intensity  Sub_metering_1  Sub_metering_2  \
41      232.93              26.2             0.0            37.0   
44      232.25              27.0             0.0            36.0   
17494   236.63              27.0             1.0            36.0   
17498   235.50              34.4             1.0            72.0   
17501   235.22              30.6             1.0            73.0   

       Sub_metering_3            Datetime  
41               17.0 2006-12-16 18:05:00  
44               17.0 2006-12-16 18:08:00  
17494   

# Task 3

### Пронормувати та стандартизувати вибраний датасет

#### Нормування

In [66]:
df_normalized = df.copy()
df_normalized[cols] = scaler_minmax.fit_transform(df_normalized[cols])

volt_norm_df = voltage_df.copy() 
volt_norm_df[cols] = scaler_minmax.fit_transform(volt_norm_df[cols])

#### Стандартизація

In [67]:
df_standart = df.copy()
df_standart[cols] = scaler_standard.fit_transform(df_standart[cols])

volt_stand_df = voltage_df.copy()
volt_stand_df[cols] = scaler_standard.fit_transform(volt_stand_df[cols])


In [68]:
print("df_normalized")
print(df_normalized[cols].head(5))
print("-----------------------------------------------------------------------------")
print("volt_norm_df")
print(volt_norm_df[cols].head(5))
print("=============================================================================")
print("df_standardized")
print(df_standart[cols].head(5))
print("-----------------------------------------------------------------------------")
print("volt_stand_df")
print(volt_stand_df[cols].head(5))


df_normalized
   Global_active_power  Global_reactive_power   Voltage  Global_intensity  \
0             0.374796               0.300719  0.376090          0.377593   
1             0.478363               0.313669  0.336995          0.473029   
2             0.479631               0.358273  0.326010          0.473029   
3             0.480898               0.361151  0.340549          0.473029   
4             0.325005               0.379856  0.403231          0.323651   

   Sub_metering_1  Sub_metering_2  Sub_metering_3  
0             0.0          0.0125        0.548387  
1             0.0          0.0125        0.516129  
2             0.0          0.0250        0.548387  
3             0.0          0.0125        0.548387  
4             0.0          0.0125        0.548387  
-----------------------------------------------------------------------------
volt_norm_df
      Global_active_power  Global_reactive_power   Voltage  Global_intensity  \
45               0.539648               

# Task 4

### Підрахувати коефіцієнт Пірсона та Спірмена для двох integer/real атрибутів.

In [69]:
spearman_corr = df_clean["Global_active_power"].corr(df_clean["Global_reactive_power"], method="spearman")

print("Кореляція між Global active power і Global reactive power:")
print(f"Коефіцієнт Пірсона - {pearson_corr:.4f}")
print(f"Коефіцієнт Спірмена - {spearman_corr:.4f}")

Кореляція між Global active power і Global reactive power:
Коефіцієнт Пірсона - 0.2470
Коефіцієнт Спірмена - 0.2693
